<a href="https://colab.research.google.com/github/garyfoohw/GA_ULD_detection/blob/main/Lock_obj_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#paramters
version = 3 #version of roboflow dataset. Latest 4 for segmentation, 5 for object detection
task = "detect" #'segment' for instance segmentation, 'detect' for object detection
epochs=100 #training epochs
yolo_pretrained_model="yolov8n.pt"
project_name = "uld_locks_cropped"
api_key=input("Enter Roboflow API Key:")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics==8.0.29 roboflow
from IPython import display
display.clear_output()

In [ ]:
import ultralytics, os, shutil
import matplotlib.pyplot as plt
import pandas as pd
import pytz
from pathlib import Path
from datetime import datetime
from IPython.display import Image

from ultralytics import YOLO
from roboflow import Roboflow

In [ ]:
ultralytics.checks()

In [ ]:
HOME = os.getcwd()

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

rf = Roboflow(api_key=api_key) 
project = rf.workspace("uld-ttkny").project(project_name)
dataset = project.version(version).download("yolov8")

## Train

In [ ]:
%cd {HOME}

model=YOLO(yolo_pretrained_model)
model.train(task=task,data=os.path.join(dataset.location,"data.yaml"),epochs=epochs,imgsz=640)

In [ ]:
#Copy best.pt to google drive

dt_now=datetime.now().astimezone(pytz.timezone('Asia/Singapore')).strftime("%Y%m%d-%H%M%S")
fn=f"{task}_{yolo_pretrained_model.split('.')[0]}_{epochs}epochs_{project_name}_v{version}_{dt_now}"

shutil.copy(f"/content/runs/{task}/train/weights/best.pt",
            f"/content/gdrive/MyDrive/ULD/{fn}.pt")

#Copy performance metrics of best.pt to google drive

shutil.copy(f"/content/runs/{task}/train/results.csv",
            f"/content/gdrive/MyDrive/ULD/results_{fn}.csv")

In [ ]:
##### TRY TO EXPORT
model.export(format="torchscript",optimize="True")

Look at train directory created

In [ ]:
!ls {HOME}/runs/{task}/train/

Look at confusion matrix of train.

In [ ]:
%cd {HOME}
try:
  Image(filename=f'{HOME}/runs/{task}/train/confusion_matrix.png', width=600)
except:
  pass

In [ ]:
#adapted from YOLOv5

def plot_results(file='path/to/results.csv', dir=''):
    # Plot training results.csv. Usage: from utils.plots import *; plot_results('path/to/results.csv')
    save_dir = Path(file).parent if file else Path(dir)
    fig, ax = plt.subplots(2, 5, figsize=(12, 6), tight_layout=True)
    ax = ax.ravel()
    files = list(save_dir.glob('results*.csv'))
    assert len(files), f'No results.csv files found in {save_dir.resolve()}, nothing to plot.'
    for f in files:
        try:
            data = pd.read_csv(f)
            s = [x.strip() for x in data.columns]
            x = data.values[:, 0]
            for i, j in enumerate([1, 2, 3, 4, 5, 8, 9, 10, 6, 7]):
                y = data.values[:, j].astype('float')
                # y[y == 0] = np.nan  # don't show zero values
                ax[i].plot(x, y, marker='.', label=f.stem, linewidth=2, markersize=8)
                ax[i].set_title(s[j], fontsize=12)
                # if j in [8, 9, 10]:  # share train and val loss y axes
                #     ax[i].get_shared_y_axes().join(ax[i], ax[i - 5])
        except Exception as e:
            print(f'Warning: Plotting error for {f}: {e}')
    ax[1].legend()
    fig.savefig(save_dir / 'results.png', dpi=200)
    plt.close()

Show training performance

In [ ]:
plot_results(f'{HOME}/runs/{task}/train/results.csv')
Image(filename=f'{HOME}/runs/{task}/train/results.png', width=1000)

Show some training segmentation results images.

In [ ]:
%cd {HOME}
try:
  Image(filename=f'{HOME}/runs/{task}/train/val_batch0_pred.jpg', width=1000)
except:
  pass

## Inference

In [ ]:
%cd {HOME}

model = YOLO(os.path.join(HOME,"runs",task,"train","weights","best.pt"))
_=model.predict(task=task,source=os.path.join(dataset.location,"test","images"), save_crop=True)

Look at the images.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/{task}/predict/crops/Raised/*.jpg'):
      display(Image(filename=image_path, height=300))
      print("\n")